In [260]:
#19016002 고가연 ass 최종 정리본, 상장일이 아닌 data 기준으로.

# 라이브러리

In [1]:
import pandas as pd
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

# !pip install finance-datareader
import os
import FinanceDataReader as fdr

import datetime as dt #https://datascienceschool.net/01%20python/02.15%20%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%97%90%EC%84%9C%20%EB%82%A0%EC%A7%9C%EC%99%80%20%EC%8B%9C%EA%B0%84%20%EB%8B%A4%EB%A3%A8%EA%B8%B0.html

import numpy as np

from tqdm import tqdm #진행바

# 코드 및 회사명 불러오기

In [2]:
code_data = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download', header=0)[0]
# code_data
code_data = code_data[['종목코드','회사명']]

def make_code(x):
    x = str(x)
    return 'A' + '0'*(6-len(x)) + x

code_data['종목코드'] = code_data['종목코드'].apply(make_code)
print(code_data[:20])

       종목코드        회사명
0   A000210         DL
1   A004840      DRB동일
2   A155660        DSR
3   A078930         GS
4   A001250      GS글로벌
5   A039570      HDC랩스
6   A294870  HDC현대산업개발
7   A011200        HMM
8   A092220        KEC
9   A016380     KG동부제철
10  A001390      KG케미칼
11  A058860       KTis
12  A011070      LG이노텍
13  A037560     LG헬로비전
14  A010060        OCI
15  A402340      SK스퀘어
16  A096770    SK이노베이션
17  A011810        STX
18  A024070     WISCOM
19  A011420    갤럭시아에스엠


# ass1 3년이상 존속 종목 선정 후 파일로 저장

## 코드: 회사명 dic 생성
```(답 확인: len(dic) == 2034)```

In [5]:
start_date = '20180101'
end_date = '20201231'
sample_code = code_data['종목코드']

dic = {}

for i in tqdm(sample_code):
    stock = fdr.DataReader(i[1:], start = start_date, end = end_date)
    if stock.size == 0:
        pass
    elif stock.index[0].date() == dt.date(2018,1,2) and stock.index[-1].date() == dt.date(2020,12,30):
        dic[i[1:]] = code_data[code_data['종목코드']==i]['회사명'].values[0]
    else:
        pass
# dic
# len(dic)

100%|██████████| 2486/2486 [03:21<00:00, 12.35it/s]


## txt 파일생성

In [ ]:
# # https://seong6496.tistory.com/108
# with open('assignment1.txt', 'w', encoding = 'UTF-8') as f:
#     for cd, nm in dic.items():
#         _ = f.write(f'{cd}\t{nm} \n')

# ass2 거래대금 >= 1조인 날짜수집

## for 문에 적용할 stock_df 함수 생성

In [7]:
def stock_df(sample_cd):
    start_date = '20180101'
    end_date = '20201231'
    sample_code = sample_cd
    stock = fdr.DataReader(sample_code, start = start_date, end = end_date)
    return stock

## 코드: 날짜list 함수생성
```(답 확인: len(dic22) == 61, txt file의 길이 == 422)```

In [8]:
f = open("./assignment1.txt", 'r')
lines = f.readlines()
dic22 = {}
for line in tqdm(lines):
    line = line.strip()
#     line[:6]
    stockdata = stock_df(line[:6])
#     stockdata
#     if (stockdata['Close']*stockdata['Volume'] >= 1_0000_0000_0000).sum() > 0:
    idx = stockdata[stockdata['Close']*stockdata['Volume'] >= 1_0000_0000_0000].index
    li = [i.date() for i in idx]
    if len(li) > 0 :
        dic22[line[:6]] = [(i.date()).strftime("%Y%m%d") for i in idx]
#     break
#     print(line)
f.close()
# dic22

100%|██████████| 2035/2035 [03:11<00:00, 10.64it/s]


## txt 파일 생성

In [62]:
# # https://seong6496.tistory.com/108
# with open('assignment2_수정.txt', 'w', encoding = 'UTF-8') as f:
#     for cd, dates in dic22.items():
#         for date in dates:
#             _ = f.write(f'{cd}\t{date} \n')

In [63]:
# 확인cell
# f = open("./assignment2_수정.txt", 'r')
# lines = f.readlines()
# len(lines)

422

# ass3 머신러닝 데이터 만들기

## 데이터 형태
```종목코드 \t 기준일(d_day) \t d-9 데이터 (open~close, volume*close), ..., d-2데이터, ..., d-day데이터 \t d+1 (next_date) 데이터 2% 상승여부 (상승:1, not상승:0```

``` 답 확인: len(dic3) == 422```

In [10]:
def stock_df2(sample_cd):
    start_date = '20180101'
    end_date = '20210104' #21년도 첫 장 개시일
    sample_code = sample_cd
    stock = fdr.DataReader(sample_code, start = start_date, end = end_date)
    return stock

In [11]:
def stock_ass3(D9, D0, cd, nextD):
    D9_date = D9
    D0_date = D0 # d-day, 기준일
    sample_code = cd
    stock1 = fdr.DataReader(sample_code, start = D9_date, end = D0_date)
    next_date = nextD
    stock2 = fdr.DataReader(sample_code, start = next_date, end = next_date)
    return stock1, stock2

In [12]:
# 날짜 형 변환해서 D0, D9 찾는 함수 # 형 변환 한 이유: D9, nextD 찾기 위해서. 그거 아니면 굳이 형변환할 필요가 없었음.
def d0d9(cd, str_d0):
    a = np.array([i.date() for i in stock_df2(cd).index])
    str_date = str_d0 #문자열로 된 날짜
    date_format = "%Y%m%d"
    D0 = dt.datetime.strptime(str_date,date_format).date() #기준일
    D9 = a[np.where(a == D0)[0][0]-9] # d-9 위치로 날짜 찾기
    nextD = a[np.where(a == D0)[0][0]+1]
    return D9, D0, nextD
d0d9('005930','20201230')

(datetime.date(2020, 12, 16),
 datetime.date(2020, 12, 30),
 datetime.date(2021, 1, 4))

In [13]:
f = open("./assignment2_수정.txt", 'r')
lines = f.readlines()
dic3 = {}
# cnt = 0
for line in tqdm(lines):
    line = line.strip()
    code = line[:6] #코드
    dday = line[7:] #날짜
    d3 = d0d9(code, dday)
    D9 = d3[0]
    D0 = d3[1]
    nextD = d3[2]
    stockdata = stock_ass3(D9, D0, code, nextD)
    stock1 = stockdata[0]
    stock2 = stockdata[1]

    stock1_1 = stock1.drop(['Change'], axis=1)
    stock1_1['거래대금'] = stock1_1['Close'] * stock1_1['Volume']

    ass3_lst = [] #한 key에 대한 value, 매번 reset하는게 맞음.
    for index, row in stock1_1.iterrows():
        l = list(row.values)
        ass3_lst.append(l)
    ass3_lst = [y for x in ass3_lst for y in x] #한 줄로 만드는 list comprehension
    
    if (stock2['Change'] >= 0.02).values[0] == True: #https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=dnjswns2280&logNo=222046755648
        ass3_lst.append(1)
    else:
        ass3_lst.append(0)

    dic3[(code,dday)] = ass3_lst
#     cnt += 1
#     if cnt % 200 == 0:
#         cnt
f.close()

100%|██████████| 422/422 [01:27<00:00,  4.81it/s]


## txt 파일 생성

In [72]:
# with open('assignment3_수정.txt', 'w', encoding = 'UTF-8') as f:
#     for cd, data in dic3.items():
#         join_list = ','.join(str(s) for s in data[:-1])
#         _=f.write(f'{cd[0]}\t{cd[1]}\t{join_list}\t{data[-1]}\n')

In [73]:
# 확인cell
# f = open("./assignment3_수정.txt", 'r')
# lines = f.readlines()
# len(lines)
# lines

422

['096770\t20200807\t132000,136000,130500,131000,576088,75467528000,132500,134000,131500,133000,560167,74502211000,132500,134500,130500,131000,454575,59549325000,131500,132500,127000,127500,667405,85094137500,128000,131000,124500,127000,656169,83333463000,128000,131000,126000,129500,741714,96051963000,132500,134500,131000,132000,561897,74170404000,133000,159000,131500,159000,6130458,974742822000,160500,171000,154000,163500,4118060,673302810000,167500,190000,165000,184000,6000468,1104086112000\t0\n',
 '003090\t20201211\t25900,26150,25250,25650,319854,8204255100,25950,32350,25250,29950,12315146,368838622700,29600,31200,29000,30400,2957185,89898424000,30400,33700,29900,30600,6019751,184204380600,32550,32700,30500,30500,1486612,45341666000,30850,32300,30500,31450,1148309,36114318050,30850,31400,29750,29950,693061,20757176950,30350,34500,30100,33650,11348392,381873390800,34700,43700,33700,42650,16742591,714071506150,41500,53900,40700,49600,22769754,1129379798400\t0\n',
 '033270\t20201222\t49